--------------------------------------------------------------------------------------

* Team member names: Nathaniel del Rosario
* Team member IDs: A17562063

--------------------------------------------------------------------------------------

# Mini-project 4, DSC 170, Winter 2024

# Suitability Modeling

This project will focus on __suitability analysis__ with raster data. Your tasks will be both conceptual-level and technical. 

There are three parts to this assignment.

1. At the conceptual level, you will define a suitability model of your choice, for an area of your choice (preferably San Diego, because we already have worked with some local data). Consult the lectures on map combination, and also see https://pro.arcgis.com/en/pro-app/latest/help/analysis/spatial-analyst/suitability-modeler/the-general-suitability-modeling-workflow.htm for a brief description of what a suitability model is. For example, you may be looking for best areas for community gardens. Such areas are often selected from underutilized land in residential land uses, with good soils, good drainage, accessible (not steep slope), etc. So you would be looking for areas with a specific type of land use/land cover, with an appropriate range of values of slope, etc. You may build additional criteria based on a range of precipitaiton values, whether the area is affected by wildfires, or has low levels of soil erosion, etc. Feel free to use the imagery layers we explored or mentioned during raster-focused lectures. Several cells in lecture notebooks contained URLs to imagery layer collections available through AGOL - but feel free to find more. Also, feel free to download additional raster layers from elsewhere (an example in lecture demonstrates how to do this from the USGS image repository, and also how to get Sentinel-2 from ESA), publish them on ArcGIS Enterprise, and use in your model. 

You can use any __two__ of the <i>map combination</i> techniques discussed during lectures. You should identify the ones you use and discuss any uncertainty issues associated with these specific map combination models. 

As the outcome of this part, you will need to: a) describe the suitability model you want to develop; b) identify the raster data layers you will use; and c) describe two of the map combination techniques you will use to derive the two suitablity maps, and their pros and cons.

2. The second part will involve implementing your suitability model using arcgis raster functions. Many of these functions are new and experimental! Examples of what works are in the lecture notebooks. Be creative!    

3. The third part will be a brief write-up comparing the two output rasters generated for the suitability models using the two map combination techniques. 

The notebook should include documentation of the steps, as usually.



**Due Date: 03/04/2024 11:59PM (Pacific Time)**

**Total Possible Points: 30 Pts**

## Task 1: Formulate a suitability or risk model (5 points)

Before finding a dataset, we will form our question in the following way:
What are the best areas for _ / best areas that satisfy condition _ ?

### Question:

What are the best locations for new student housing in Berkeley?

### Data
- [Berkeley zoning data](https://data.cityofberkeley.info/browse?q=zoning&sortBy=relevance)
- [Population By Council District](https://geodata.lib.berkeley.edu/catalog/berkeley-s7wq14)
- [GeoData](https://geodata.lib.berkeley.edu/?utf8=%E2%9C%93&search_field=all_fields&q=berkeley)
- [Alameda County Open Data](https://data.acgov.org/search?collection=Dataset&source=AlamedaCounty.CA.US)
- [Fire Hazard Raster](https://ucsdonline.maps.arcgis.com/apps/mapviewer/index.html?layers=cdedfa4417f54ae0a274c78f7c9b8c8e)
- [Census Tracts](https://data.census.gov/allq=alameda%20&t=Housing:Housing%20Units:Renter%20Costs&g=050XX00US06001$1400000)
- [Berkeley Crime Data](https://ucsdonline.maps.arcgis.com/apps/mapviewer/index.html?webmap=9f67de4e3a0d49c39ae07034f5c32966)
- [Berkeley Bus Routes](https://ucsdonline.maps.arcgis.com/apps/mapviewer/index.html?webmap=761a98de8a1a435aad5f229692c5d229)
- [Berkeley Bike Lanes](https://ucsdonline.maps.arcgis.com/apps/mapviewer/index.html?layers=e1453a81e05f45dd9c44699c5b1a40b7)
- [Berkeley Bike Parking](https://ucsdonline.maps.arcgis.com/apps/mapviewer/index.html?webmap=9fa9fa7139584d359a0542cc4561d427)

### Important Features
- Population density
- Proximity to bike paths / bus routes
- Is it in a heat island?
- Proximity to frequent crime scenes 


## Task 2: Implement the model (20 points)

In [1]:
# Imports, etc.
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

import rasterio
from rasterio.features import rasterize
from arcgis.raster.analytics import convert_feature_to_raster

import arcgis
from arcgis.gis import GIS
from arcgis import geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureLayerCollection, FeatureSet, FeatureCollection, FeatureLayer
from arcgis.features.use_proximity import create_buffers
from arcgis.raster import ImageryLayer
from IPython.display import display
import os

In [2]:
gis = GIS("https://ucsdonline.maps.arcgis.com/home/index.html", "dsc170wi24_8", "NDRCZBH@499!")

In [3]:
berkeley_map = gis.map('Berkeley, CA')
berkeley_map

MapView(layout=Layout(height='400px', width='100%'))

In [4]:
# List imagery layers to be used in your model. 
# This cell should contian layer definitions.

# fire risk raster
fire_risk_raster = gis.content.get('6229ec92b2894b9ca718c9e7163bbd7b')

# AC Transit Route, plus an additional bus route
bus_routes1_fl = gis.content.get('1b6d51b614bd47a99efd60cb64ddcdc5')
bus_routes2_fl = gis.content.get('aba7cd6ad57f4102a8af2264e29a5119')

# crime data
crime_fl = gis.content.get('8f4514aca02f417eb6004f03b832d6a9')
vulnerability_fl = gis.content.get('31b839e6391741ccb7887b7f630add63')

# zoning layer
zoning_fl = gis.content.get('9b6611b77cc047ea813ffe7346b1637a') # this is just here for visualization reference

#item_properties = {
#    "type": "GeoJson",
#    "title": "Berkeley Zoning Districts"
#}
#geojson_item = gis.content.add(item_properties, 'ZoningDistricts.geojson')
#zoning_layer = geojson_item.publish()
#zoning_layer

In [5]:
# play around until all layers show: there should be bus routes, blue zoning, and yellow-purple heatmap
vulnerability_fl.opacity = 0.8
zoning_fl.opacity = 0.2
bus_routes1_fl.opacity = 1
bus_routes2_fl.opacity = 1
berkeley_map.add_layer(vulnerability_fl)
berkeley_map.add_layer(zoning_fl)
berkeley_map.add_layer(bus_routes1_fl)
berkeley_map.add_layer(bus_routes2_fl)
berkeley_map

MapView(layout=Layout(height='400px', width='100%'))

In [6]:
# Convert Feature Layers to Rasters
# bus_routes_raster = convert_feature_to_raster(bus_routes1_fl, output_cell_size={"distance": 60,"units": "meters"},
#                                 gis=gis) 59ee8b2ae4844aa2926ab4112309d4d9
#vulnerability_raster = convert_feature_to_raster(vulnerability_fl, 
#                                         output_cell_size={"distance": 60,"units": "meters"}, 
#                                         context={"extent": {"xmin": -13619445.9854612, "ymin": 4550330.01498604, "xmax": -13608121.1308526, "ymax": 4576582.91869095, "spatialReference": {"wkid": 3857}}}, gis=gis)
# 9ec715bc53a04267b477dc78cce07b59

# retrieve rasters
bus_routes_raster = gis.content.get('59ee8b2ae4844aa2926ab4112309d4d9')
vulnerability_raster = gis.content.get('9ec715bc53a04267b477dc78cce07b59')

In [7]:
# Derive the area of interest (AOI) and its geometry and extent. 
# The smaller the area the better (so that you don't run into raster size limitations)

"""
Crime Data Extent for AOI

XMIN: -13619445.9854612
XMax: -13608121.1308526
YMin: 4550330.01498604
YMax: 4576582.91869095
"""

from arcgis.geocoding import geocode

# Use geocoding to get the location of the study area in the spatial reference of the input data for the analysis.
study_area_gcd = geocode(address='Berkeley, CA')

# 4 corners: [xmin, ymin], [xmin, ymax], [xmax, ymin], [xmax, ymax]
study_area_geom = {'rings': [[[-13619445.9854612, 4550330.01498604], [-13608121.1308526, 4550330.01498604], [-13608121.1308526, 4550330.01498604], [-13608121.1308526, 4576582.91869095]]], 
                    'spatialReference': {'wkid': 3857, 'latestWkid': 3857}}
#study_area_extent = study_area_gcd[0]['extent']
study_area_extent = {'xmin': -13619445.9854612, 'ymin': 4550330.01498604, 'xmax': -13608121.1308526, 'ymax': 4576582.91869095}

study_area_polygon = {
    "type": "polygon",
    "rings": [
        [
            [study_area_extent['xmin'], study_area_extent['ymin']],
            [study_area_extent['xmax'], study_area_extent['ymin']],
            [study_area_extent['xmax'], study_area_extent['ymax']],
            [study_area_extent['xmin'], study_area_extent['ymax']],
            [study_area_extent['xmin'], study_area_extent['ymin']]
        ]
    ]
}

# Get the geographic extent of the study area.
# This extent will be used for displaying the input data and output results.


## Typical steps for preparing a raster layer for suitability analysis:
 - retrieve the layer
 - specify area of interest (study area), e.g. by retrieving a named polygon and getting its extent in an explicit CRS
 - clip the layer to the area of interest
 - define categories to be shown in the output (these may be suitability classes) (make sure these classes actually exist)
 - define a colormap for these classes
 - remap the layer clip to these categories in this colormap
 

### Name the two map combination techniques you will use to combine the data and describe their pros and cons

$$F1 = 1 - \frac{1}{\sum_{fl}(\text{feature_layers})} = 1 - \frac{1}{\text{fire_risk} + \text{vulnerability_data}} $$

The scales of the fire risk and vulnerablility layers were similar so I decided to not normalize, and instead just have an interpretable score for evaluation that is also between 0 and 1. The pros of this is that it requires less preprocessing and provides a quick cumulative summed value of how high at risk an area is. I'll break down my design choice further.

Berkeley has a history of burglary and crime around campus, and therefore the lower at risk a specific street section is to crime, the more favorable for students it is. Additionally, a lot of student dorms do not have air conditioning, which in the fall and summer, can give bad living conditions as well as pose a risk for fire if the area is too hot around. Therefore it is important that we have a metric which points in the direction of high risk and low risk. I decided to create a score designed around the inverse of the sum of these two layers, which allows for high risk values to be larger (conceptually this is 1 - $\frac{1}{\text{large/small sum}}$) and lower risk values to be higher. Another potential way of doing this was the following: 

$$-log(\frac{1}{\text{fire_risk} + \text{vulnerability_data}})$$

However these values would have to be normalized again in order to be between 0 and 1.


$$ F2 = \frac{alpha}{\text{fire_risk} + \text{vulnerability_data}} + (1 - \alpha) * \text{bus_routes} $$
$$ \alpha = 0.5 \text{ (however it is tunable depending on how much you want to emphasis importance of bus usage among students)}$$

The design choice behind this function was to just have a raw score and then add bias term based on how important bus route buffer proximity should be in the decision for housing, which would only be added if the location was within a bus route buffer.

The pros behind this score is that it has a tunable parameter to which you can emphasize safety and risk more, and convenience more. In practice, this can help on tiebreakers between a set of suitable locations to live, where one location may be farther from a bus buffer but safer, and one location is closer but in a high traffic area more at risk to crimes. In both cases you could choose alphas bias risk more, bus routes more, or both evenly ($\alpha=0.5$) and then compare these combinations in a table to see which one is most suitable for the given context.

The con is that it would require trying many different alpha values and seeing how the scores change. This would need to be done to ensure that no biases occur in "picking / being selective with alphas" to support risk averseness or transportation convenience more.

In [9]:
# Prepare your input layers for map combination: clip to AOI, remap/normalize, visualize the layers. 
from arcgis.raster.functions import clip

bus_routes_raster_clip = clip(bus_routes_raster, study_area_geom)
vulnerability_raster_clip = clip(vulnerability_raster, study_area_geom)
fire_risk_raster_clip = clip(fire_risk_raster, study_area_geom)

RecursionError: maximum recursion depth exceeded

In [10]:
# Generate a composite raster layer for your first map combination technique
# F1 = 1 - \frac{1}{\sum_{fl}(\text{feature_layers})} = 1 - \frac{1}{\text{fire_risk} + \text{vulnerability_data}} 

In [11]:
# Generate a composite raster layer for your second map combination technique
# F2 = \frac{alpha}{\text{fire_risk} + \text{vulnerability_data}} + (1 - \alpha) * \text{bus_routes}

# the second function differs as we add another feature, bus_routes which is a 0 or 1 value:
# if there is a bus route nearby, we add 1, else the score stays the same

## Task 3: Compare the results (5 points)
... and describe how different combination techniques resulted in different outputs (or not.) 

Comparison:
When comparing the outputs of the two combination techniques, we would expect to see differences in the prioritization of locations based on their risk and convenience factors:

F1 would prioritize locations with lower combined risk scores, irrespective of their proximity to bus routes. It provides a straightforward assessment of overall risk without considering other contextual factors.

F2 would provide a more nuanced evaluation, considering both risk factors and the convenience of bus routes. The parameter α allows for flexibility in adjusting the balance between safety and convenience, leading to potentially different outputs based on the chosen value.

Since the functions were still a little similar, with one including bus route proximity and one not including bus route proximity, the difference in outputs lies in areas around bus routes as well as the strength of alpha for the bus route feature. Because the second function could be tweaked to emphasize importance of this proximity to a bus route the scores will vary based on how alpha is tweaked to favor bike route proximity or risk averseness. In the scope of planning a new student housing, the reasoning for a function that excludes and includes bus route proximity is that some students may not bus to school/if the housing was within a few blocks to campus, the priority / need for a bus in this context would not really be important compared to housing that is farther (in this case, more West towards I-80 and Telegraph Ave & Beyond).

In [ ]:
## Timekeeping
# Please let us know how much time you spent on this project, in hours: 
# (we will only examine distributions and won't look at individual responses)

assignment_timespent = 8 # mostly because I was having a lot of difficulty converting from FL to Raster